이번엔 바로 학습된 모델의 파라미터를 받아와서 그 모델을 평가해보고, 얼굴을 만들어보기

In [ ]:
!gdown --id liUkepsRdaADfCnczvx2-jkcT2bHLUyPd
# 이번엔 학습 파라미터를 바로 받아보자

In [ ]:
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [ ]:
def show(img, renorm=False, nrow=8, interpolation='bicubic'):
  if renorm:
    img = img*0.5 + 0.5
  img_grid = torchvision.utils.make_grid(img, nrow=nrow).numpy()
  plt.figure()
  plt.imshow(np.transpose(img_grid, (1,2,0)), interpolation=interpolation)
  plt.axis('off')
  plt.show()

In [ ]:
root = 'data_faces/img_align_celeba'
img_list = os.listdir(root)
print(len(img_list))

In [ ]:
class VAE(nn.Module):
  def __init__(self, image_size=128, latent_dim=512):
    super(VAE, self).__init__()

    # Encoder
    self.encoder = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.Flatten()
    )
    # 평균, 분산을 확인할 수 있는 파라미터 정의
    self.fc_mu = nn.Linear(256*(image_size // 16) * (image_size // 16), latent_dim)
    self.fc_logvar = nn.Linear(256*(image_size // 16) * (image_size // 16), latent_dim)

    # Decoder
    self.decoder_input = nn.Linear(latent_dim, 256 * (image_size // 16) * (image_size // 16))
    self.decoder = nn.Sequential(
        nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),
        nn.Sigmoid()
    )
    self.image_size = image_size

  def encoder(self, x):
    x = self.encoder(x)
    mu, logvar = self.fc_mu(x), self.fc_log_var(x)
    return mu, logvar

  def reparameterize(self, mu, logvar):
    std = torch.exp(0.5*logvar)
    eps = torch.randn_like(std)
    return mu * eps * std

  def decoder(self, z):
    x = self.decoder_input(z)
    x = x.view(-1, 256, (self.image_size // 16), (self.image_size // 16))
    x = self.decoder(x)
    return x

  def forward(self, x):
    mu, logvar = self.encoder(x)
    z = self.reparameterize(mu, logvar)
    reconstructed_x = self.decoder(z)
    return reconstructed_x, mu, logvar


In [ ]:
re_size = 128

transform = transforms.Compose([
    transforms.Resize(size=(re_size, re_size), interpolation=Image.BICUBIC),
    transforms.ToTensor()
])

batch_size = 64
celeba_data = datasets.ImageFolder('./data_faces', transform=transform)
celeba_loader = DataLoader(celeba_data, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
model = VAE().to(device)
model.load_state_dict(torch.load('celeba_vae.pth'))
model.eval()
# 이미 훈련된 모델을 쓰기 때문에 훈련을 할 필요 없이 해당 모델을 사용하면 됨
# 모델은 훈련하기 위해 기본적으로 gradient를 계산하는데, 평가 형태로 두면 gradient를 계산하지 않게 되어 모델의 연산이 훨씬 더 빠르게 됨


In [ ]:
# 그림 생성
batch, _ = next(iter(celeba_loader))


In [ ]:
reconstruction, _, _ = model(batch.cuda())
display_grid(batch, figsize=(8,8))
plt.savefig('vae_training_data.png', bbox_inches='tight', pad_inches=0, dpi=156)
plt.show()
display_grid(reconstruction.to('cpu'), figsize=(8,8))
plt.savefig('vae_reconstruction.png', bbox_inches='tight', pad_inches=0, dpi=156)
plt.show()

In [ ]:
#데이터 원본과 복원된 그림을 보면 복원된 그림의 품질이 아주 선명하진 않기도 함, 해당 그림의 품질을 더 높일 수도 있음
sampled_latents = torch.randn(len(batch), 512).cuda()

with torch.no_grad():
  reconstructed_x = model.decode(sampled_latents)

display_grid(reconstructed_x.detach().clip(0,1).cpu(), figsize=(8,8))

In [ ]:
# Vector Interpolation
batch, _ = next(iter(celeba_loader))
display_grid(batch, figsize=(8,8))


In [ ]:
#
target_images = [0, 2]
reconstructed_x, mu, logvar = model(batch[target_images].cuda())

In [ ]:
latents = model.reparameterize(mu,logvar).detach().cpu().numpy()

vectors = []
ratios = np.linspace(0,1, mum=10)

# Vector interpolation
for ratio in ratios:
  v = (1.0 - ratio) * latents[0]+ ratio * latents[1]
  vectors.append(v)

vectors = np.asarray(vectors)

with torch.no_grad():
  reconstructed_x = model.decode(torch.from_numpy(vectors).cuda()).permute(0,2,3,1).detach().clip(0,1).cpu().numpy()


  fig, axes = plt.subplots(1,10, figsize=(20, 2))
  for i in range(10):
    axes[i].imshow(reconstructed_x[i])
    axes[i].axis('off')
  plt.show()

In [ ]:
# Image Attributes Modification
# 이것을 적용하려면 그림에 어떤 속성이 있는지를 같이 불러들여서 그림과 그 그림의 속성을 같이 맞추어 주어야 함!
# 그래서 아래와 같이 데이터 셋을 정의하고,

class CelebADataset(Dataset):
  def __init__(self, data, labels, classes, transform=None):
    self.data = data
    self.labels = labels
    self.classes = classes
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    img = Image.open(os.path.join('data_faces', 'img_align_celeba', self.data[idx])).convert('RGB')
    label = torch.Tensor(self.labels[idx, [39, 20, 22, 2, 31, 15]].astype('uint8'))
    if self.transform:
      img = self.transform(img)
    sample = {'images': img, 'labels': label}
    return sample


In [ ]:
df = pd.read_csv('list_attr_celeba.csv')
df.head()
# 어떤 특성이 있는지 csv로 어느 그림에 어느 속성들이 들어있는지 확인한 뒤에

In [ ]:
values = df.values

values[:, 1:] += 1
values[:, 1:]/=2

# -1,과 1로 된 값을 0과 1로 맞춰주기 위해 위와 같이 처리하는 것

classes = df.columns[1:]

In [ ]:
values.shape

In [ ]:
classes.shape

In [ ]:
classes[[39, 20, 22, 2, 31, 15]]
# 특징들 중에 특정 타입만 뽑아보기! 여섯개의 특징을 확인

In [ ]:
np.mean(values[:, 15])

In [ ]:
att_idxs = [39, 20, 22, 2, 31, 15]
att_names = classes[att_idxs]
attnames

In [ ]:
att_idx=20+1

In [ ]:
if not os.path.exists(os.path.join('outputs', classes[att_idx-1]+ 'neg_pos.npy')):
  neg_data = CelebADataset(values[values[:, att_idx]==0, 0], values[values[:, att_idx]==0, 1:], classes, transform)
  pos_data = CelebADataset(values[values[:, att_idx]==1, 0], values[values[:, att_idx]==1, 1:], classes, transform),

  neg_loader = DataLoader(neg_data, batch_size=batch_size)
  pos_loader = DataLoader(pos_data, batch_size=batch_size)

  print('Negative latent')
  neg_latent = torch.zeros(512)
  cpt = 0
  with torch.no_grad():
    for batch_idx, data in enumerate(neg_loader):
      imgs, labels = data['images'].float().to(device), data['labels'].long().to(device)

      recon_img, mu, logvar = model(imgs)
      neg_latent += torch.sum(model.reparameterize(mu, logvar).detach().cpu(), 0)
      cpt+= 64
      if batch_idx % 100 == 0:
        print(f'Step[{batch_idx+1}/{len(neg_loader)}]')

  neg_latent/=cpt
  print('Positive latent')
  pos_latent = torch.zeros(512)
  cpt=0
  with torch.no_grad():
    for batch_idx, data in enumerate(pos_loader):
      imgs, labels = data['images'].float().to(device), data['labels'].long().to(device)

      recon_img, mu, logvar = model(imgs)
      pos_latent += torch.sum(model.reparameterize(mu, logvar).detach().cpu(), 0)
      cpt+= 64
      if batch_idx % 100 == 0:
        print(f'Step[{batch_idx+1}/{len(neg_loader)}]')
  pos_latent/=cpt
  V = (pos_latent-neg_latent).detach().cpu().numpy()[0]
  V /= np.linalg.norm(V)
  np.save(os.path.join('outputs', classes[att_idx-1]+'neg_pos.npy'), V)
else:
  V = np.load(os.path.join('outputs', classes[att_idx=1]+'neg_pos.npy'))

In [ ]:
label_cls = np.random.choice(2)
idxs = np.where(values[:, att_idx]==label_cls)[0]
sample_idx = idxs[np.random.choice(len(idxs))]

img = Image.open(os.path.join('data_faces', 'img_align_celeba', values[sample_idx, 0])).convert('RGB')
img = transform(img).unsqueeze(0).float().cuda()

with torch.no_grad():
  reconstructions, mu, logvar = model(img)
  reconstructions = reconstructions.datach().cpu().clip(0, 1)

plt.subplot(121)
plt.imshow(img.permute(0,2,3,1).cpu().numpy()[0])
plt.axis('off')
plt.title('Original')
plt.subplot(122)
plt.imshow(reconstructions.permute(0,2,3,1).cpu().numpy()[0])
plt.axis('off')
plt.title('Original '+ classes[att_idx-1]+'%d'%(label_cls))


In [ ]:
latents = model.reparameterize(mu, logvar).detach().cpu().numpy()

vectors = []
ratios = np.linspace(0, 1, num=10)

for ratio in ratios:
  if label_cls==0:
    v = latents[0] + ratio * V
  else:
    v = latents[0] - ratio * V

  vectors.append(v)

vectors = np.asarray(vectors)

reconstructed_x = model.decode(torch.from_numpy(vectors).cuda()).permute(0,2,3,1).detach().clip(0,1).cpu().numpy()

#
fig, axes = plt.subplot(1, 10, figsize=(20, 20))
for i in range(10):
  axes[i].imshow(reconstructed_x[i])
  axes[i].axis('off')

plt.show()